In [62]:
import pandas as pd
import os
import regex as re
import copy

In [68]:
# Read the crop.csv file into a DataFrame
data = pd.read_csv('../data/crop_data/corn.csv')

In [85]:
data.head()

# Create a new empty DataFrame
cleaned_df = pd.DataFrame()


In [100]:
cleaned_df = copy.deepcopy(data)


cleaned_df["PriceIsNan"] = data["PRICE RECEIVED in $ / BU"].apply(lambda x: not bool(re.match(r'^[0-9]', str(x))))
cleaned_df["AreaisNan"] = data["AREA PLANTED in ACRES"].apply(lambda x: not bool(re.match(r'^[0-9]', str(x))))

# Filter rows where "AREA PLANTED in ACRES" is greater than 100,000
# First remove commas from the values and convert to numeric
cleaned_df = data[cleaned_df['AreaisNan'] == False]
cleaned_df["AREA PLANTED in ACRES"] = cleaned_df["AREA PLANTED in ACRES"].str.replace(',', '').astype(int)
cleaned_df = cleaned_df[cleaned_df["AREA PLANTED in ACRES"] > 3000000]

C:\Users\mansu\AppData\Local\Temp\ipykernel_14436\3110129329.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df["AREA PLANTED in ACRES"] = cleaned_df["AREA PLANTED in ACRES"].str.replace(',', '').astype(int)


In [106]:
cleaned_df = copy.deepcopy(data)

cleaned_df = data[data['COMMODITY'] == "CORN, GRAIN"]
cleaned_df = cleaned_df.drop(["PRODUCTION in TONS", "COMMODITY", "ASD CODE", "COUNTY ANSI", "AREA PLANTED in ACRES", "YIELD in TONS / ACRE", "STATE ANSI"], axis=1)


In [107]:
cleaned_df.head(50)

,YEAR,LOCATION,REFERENCE PERIOD,AREA HARVESTED in ACRES,PRODUCTION in BU,YIELD in BU / ACRE,PRODUCTION in $,PRICE RECEIVED in $ / BU
1210,2000,ALABAMA,YEAR,"165,000","10,725,000",65,"23,166,000",
1211,2000,ALABAMA,MARKETING YEAR,,,,,2.16
1212,2000,ARIZONA,YEAR,"33,000","6,468,000",196,"17,981,000",
1213,2000,ARIZONA,MARKETING YEAR,,,,,2.78
1214,2000,ARKANSAS,YEAR,"175,000","22,750,000",130,"39,813,000",
1215,2000,ARKANSAS,MARKETING YEAR,,,,,1.75
1216,2000,CALIFORNIA,YEAR,"205,000","34,850,000",170,"85,034,000",
1217,2000,CALIFORNIA,MARKETING YEAR,,,,,2.44
1218,2000,COLORADO,YEAR,"1,150,000","144,900,000",126,"301,392,000",
1219,2000,COLORADO,MARKETING YEAR,,,,,2.08


In [21]:
# Remove rows where isNan is True
cleaned_df = cleaned_df[cleaned_df['isNan'] == False]


In [78]:
cleaned_df.head(50)

,YEAR,LOCATION,STATE ANSI,ASD CODE,COUNTY ANSI,REFERENCE PERIOD,COMMODITY,AREA HARVESTED in ACRES,PRODUCTION in BU,YIELD in BU / ACRE,AREA PLANTED in ACRES,PRODUCTION in $,PRODUCTION in TONS,YIELD in TONS / ACRE,PRICE RECEIVED in $ / BU
0,2000,ALABAMA,01,,,YEAR,CORN,,,,230000.0,,,,
2,2000,ARKANSAS,05,,,YEAR,CORN,,,,180000.0,,,,
3,2000,CALIFORNIA,06,,,YEAR,CORN,,,,540000.0,,,,
4,2000,COLORADO,08,,,YEAR,CORN,,,,1350000.0,,,,
6,2000,DELAWARE,10,,,YEAR,CORN,,,,165000.0,,,,
8,2000,GEORGIA,13,,,YEAR,CORN,,,,360000.0,,,,
9,2000,IDAHO,16,,,YEAR,CORN,,,,195000.0,,,,
10,2000,ILLINOIS,17,,,YEAR,CORN,,,,11200000.0,,,,
11,2000,INDIANA,18,,,YEAR,CORN,,,,5700000.0,,,,
12,2000,IOWA,19,,,YEAR,CORN,,,,12300000.0,,,,


In [50]:
del cleaned_df